In [ ]:
import os 
from PIL import Image 
from torch.utils.data import Dataset
import numpy as np

In [ ]:
#DataSet

import os
from PIL import Image
from torch.utils.data import Dataset
import numpy as np

class CarvanaDataset(Dataset):
    def __init__(self, image_dir, mask_dir, transform=None):
        self.image_dir = image_dir
        self.mask_dir = mask_dir
        self.transform = transform
        self.images = os.listdir(image_dir)

    def __len__(self):
        return len(self.images)

    def __getitem__(self, index):
        img_path = os.path.join(self.image_dir, self.images[index])
        mask_path = os.path.join(self.mask_dir, self.images[index].replace(".jpg", "_mask.gif"))
        image = np.array(Image.open(img_path).convert("RGB"))
        mask = np.array(Image.open(mask_path).convert("L"), dtype=np.float32)
        mask[mask == 255.0] = 1.0

        if self.transform is not None:
            augmentations = self.transform(image=image, mask=mask)
            image = augmentations["image"]
            mask = augmentations["mask"]

        return image, mask

In [ ]:
! pip install albumentations==0.4.6

In [ ]:
import torch 
import torch.nn as nn


def double_conv(in_c, out_c):
  conv = nn.Sequential(
      nn.Conv2d(in_c, out_c, kernel_size=3),
      nn.ReLU(inplace=True),
      nn.Conv2d(out_c, out_c, kernel_size=3),
      nn.ReLU(inplace=True)
  )
  return conv

def crop_img(tensor, target_tensor):
  target_size = target_tensor.size()[2]
  tensor_size = tensor.size()[2]
  delta = tensor_size - target_size
  delta = delta//2
  return tensor[: , : , delta: tensor_size-delta, delta: tensor_size-delta]





class UNet(nn.Module):
    def __init__(self):
      
      super(UNet, self).__init__()

      self.max_pool_2x2 = nn.MaxPool2d(kernel_size = 2, stride = 2)
      self.down_conv_1 = double_conv(1, 64)
      self.down_conv_2 = double_conv(64, 128)
      self.down_conv_3 = double_conv(128, 256)
      self.down_conv_4 = double_conv(256, 512)
      self.down_conv_5 = double_conv(512, 1024)

      self.up_trans_1= nn.ConvTranspose2d(in_channels=1024, out_channels=512, kernel_size=2, stride=2 )
      self.up_conv_1= double_conv(1024, 512)
      self.up_trans_2 = nn.ConvTranspose2d(in_channels=512, out_channels=256, kernel_size=2, stride=2 )
      self.up_conv_2= double_conv(512, 256)
      self.up_trans_3 = nn.ConvTranspose2d(in_channels=256, out_channels=128, kernel_size=2, stride=2 )
      self.up_conv_3= double_conv(256, 128)
      self.up_trans_4 = nn.ConvTranspose2d(in_channels=128, out_channels=64, kernel_size=2, stride=2 )
      self.up_conv_4= double_conv(128, 64)

      self.out = nn.Conv2d(in_channels=64, out_channels=2, kernel_size=1)




    def forward(self, image):
      #encoder part 
      # bs, c, h, w
      x1 = self.down_conv_1(image)  #
      x2= self.max_pool_2x2(x1)
      x3= self.down_conv_2(x2)    # 
      x4= self.max_pool_2x2(x3)
      x5= self.down_conv_3(x4)   #
      x6= self.max_pool_2x2(x5)
      x7= self.down_conv_4(x6)   # 
      x8= self.max_pool_2x2(x7)
      x9= self.down_conv_5(x8)

    #decoder part
      x = self.up_trans_1(x9)
      y = crop_img(x7, x)
      x = self.up_conv_1(torch.cat([x,y], 1))

      x = self.up_trans_2(x)
      y = crop_img(x5, x)
      x = self.up_conv_2(torch.cat([x,y], 1))

      x = self.up_trans_3(x)
      y = crop_img(x3, x)
      x = self.up_conv_3(torch.cat([x,y], 1))

      x = self.up_trans_4(x)
      y = crop_img(x1, x)
      x = self.up_conv_4(torch.cat([x,y], 1))

      x = self.out(x)
      # print(x.size())
      # return x

if __name__ == "__main__":

  image =torch.rand((1, 1, 572, 572))
  model = UNet()
  print(model(image))


None


In [ ]:
import torch
import albumentations as A
from albumentations.pytorch import ToTensorV2
from tqdm import tqdm
import torch.nn as nn
import torch.optim as optim
# from model import UNet
# from utils import (
#     load_checkpoint,
#     save_checkpoint,
#     get_loaders,
#     check_accuracy,
#     save_predictions_as_imgs,
# )

# Hyperparameters etc.
LEARNING_RATE = 1e-4
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
BATCH_SIZE = 16
NUM_EPOCHS = 3
NUM_WORKERS = 2
IMAGE_HEIGHT = 160  # 1280 originally
IMAGE_WIDTH = 240  # 1918 originally
PIN_MEMORY = True
LOAD_MODEL = False
TRAIN_IMG_DIR = "data/train_images/"
TRAIN_MASK_DIR = "data/train_masks/"
VAL_IMG_DIR = "data/val_images/"
VAL_MASK_DIR = "data/val_masks/"

def train_fn(loader, model, optimizer, loss_fn, scaler):
    loop = tqdm(loader)

    for batch_idx, (data, targets) in enumerate(loop):
        data = data.to(device=DEVICE)
        targets = targets.float().unsqueeze(1).to(device=DEVICE)

        # forward
        with torch.cuda.amp.autocast():
            predictions = model(data)
            loss = loss_fn(predictions, targets)

        # backward
        optimizer.zero_grad()
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        # update tqdm loop
        loop.set_postfix(loss=loss.item())


def main():
    train_transform = A.Compose(
        [
            A.Resize(height=IMAGE_HEIGHT, width=IMAGE_WIDTH),
            A.Rotate(limit=35, p=1.0),
            A.HorizontalFlip(p=0.5),
            A.VerticalFlip(p=0.1),
            A.Normalize(
                mean=[0.0, 0.0, 0.0],
                std=[1.0, 1.0, 1.0],
                max_pixel_value=255.0,
            ),
            ToTensorV2(),
        ],
    )

    val_transforms = A.Compose(
        [
            A.Resize(height=IMAGE_HEIGHT, width=IMAGE_WIDTH),
            A.Normalize(
                mean=[0.0, 0.0, 0.0],
                std=[1.0, 1.0, 1.0],
                max_pixel_value=255.0,
            ),
            ToTensorV2(),
        ],
    )

    model = UNet(in_c= 3, out_c=1).to(DEVICE)
    loss_fn = nn.BCEWithLogitsLoss()
    optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

    train_loader, val_loader = get_loaders(
        TRAIN_IMG_DIR,
        TRAIN_MASK_DIR,
        VAL_IMG_DIR,
        VAL_MASK_DIR,
        BATCH_SIZE,
        train_transform,
        val_transforms,
        NUM_WORKERS,
        PIN_MEMORY,
    )

    if LOAD_MODEL:
        load_checkpoint(torch.load("my_checkpoint.pth.tar"), model)


    check_accuracy(val_loader, model, device=DEVICE)
    scaler = torch.cuda.amp.GradScaler()

    for epoch in range(NUM_EPOCHS):
        train_fn(train_loader, model, optimizer, loss_fn, scaler)

        # save model
        checkpoint = {
            "state_dict": model.state_dict(),
            "optimizer":optimizer.state_dict(),
        }
        save_checkpoint(checkpoint)

        # check accuracy
        check_accuracy(val_loader, model, device=DEVICE)

        # print some examples to a folder
        save_predictions_as_imgs(
            val_loader, model, folder="saved_images/", device=DEVICE
        )


if __name__ == "__main__":
    main()

TypeError: ignored